In [9]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tqdm import trange

In [10]:
# 加载鸢尾花数据集
iris = datasets.load_iris()
df = pd.DataFrame(iris.data)

# 加载鸢尾花数据集
x = iris.data
y = iris.target

# 加载鸢是三分类，将它转为二分类
# y = np.array([1 if i == 1 else 0 for i in y])
# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)


In [11]:
class Model:
    def __init__(self, train_data, train_label, num):
        self.train_data = train_data  # 训练集数据
        self.train_label = train_label  # 训练集标记
        self.num = num  # 多少个基本分类器
        # m为样本容量，n为特征数量
        m, n = np.shape(train_data)
        self.m = m  # m为样本容量
        self.n = n  # n为特征数量
        self.w = np.full(m, fill_value=1 / m)
        self.tree_arr = []
        self.alpha_arr = []

    def estimate(self, feature, divide, rule):
        if rule == 'H1L-1':
            H = 1
            L = -1
        else:
            H = -1
            L = 1

        error = 0
        predict_arr = []

        num = len(feature)

        # 遍历每个样本，判断样本的该特征值是否大于divide
        # 并预测相应的类
        for i in range(num):
            if feature[i] > divide:
                predict = H
            else:
                predict = L

            predict_arr.append(predict)

            # 如果预测值与标记不符
            # 则error + 该分类错误样本所对应的权值
            if predict != self.train_label[i]:
                error += self.w[i]

        return predict_arr, error

    def create_single_boosting_tree(self):
        # 错误的样本数
        error = 1
        tree_dict = {}
        for i in range(self.n):
            feature = self.train_data[:, i]

            for divide in [-0.5, 0.5, 1.5]:
                for rule in ['H1L-1', 'H-1L1']:
                    predict_arr, e = self.estimate(feature, divide, rule)

                    if e < error:
                        error = e
                    tree_dict['feature'] = i
                    tree_dict['divide'] = divide
                    tree_dict['rule'] = rule
                    tree_dict['error'] = e
                    tree_dict['PredictArr'] = predict_arr
        return tree_dict

    def create_boosting_tree(self):
        for N in trange(self.num):
            tree_dict = self.create_single_boosting_tree()
            self.tree_arr.append(tree_dict)
            
            e = tree_dict["error"]
            alpha = np.log((1 - e) / e) / 2
            self.alpha_arr.append(alpha)
            
            gxi=tree_dict["PredictArr"]
    
            exp = np.exp(-1*alpha*self.train_label*gxi)
            
            Z = np.dot(self.w, exp)
            
            self.w = self.w * exp / Z
            
    
    def fit(self):
        self.create_boosting_tree()

    def predict(self, feature, divide, rule, x):
        # 用于之后预测在该情况下每个样本属于哪个类
        if rule == 'H1L-1':
            H = 1
            L = -1
        else:
            H = -1
            L = 1
    
        # 根据特征值的大小返回相应预测值
        if x[feature] > divide:
            return H
        else:
            return L
    def test(self, test_data, test_label):
        error = 0
        for i in trange(len(test_label)):
            xi = test_data[i]
            yi = test_label[i]
            
            result = 0
            
            for j in range(self.num):
                tree = self.tree_arr[j]

                feature = tree['feature']
                divide = tree['divide']
                rule = tree['rule']

                weak_result = self.predict(feature, divide, rule, xi)

                alpha = self.alpha_arr[j]
                result += alpha * weak_result

            final_result = np.sign(result)

            # 如果分类错误，errorCnt + 1
            if final_result != yi:
                error += 1
        
        acc = 1- error / len(test_label)

        return acc


In [12]:
adr = Model(x_train, y_train, 1000)
adr.fit()

acc = adr.test(x_test, y_test)
print("acc", acc)

 14%|█▍        | 140/1000 [00:00<00:00, 1329.42it/s]

组合1棵回归树后的提升树平方损失为:0.6666666666666665
组合2棵回归树后的提升树平方损失为:0.5300541098297
组合3棵回归树后的提升树平方损失为:0.5089036314641198
组合4棵回归树后的提升树平方损失为:0.5028464855179127
组合5棵回归树后的提升树平方损失为:0.5009303405199879
组合6棵回归树后的提升树平方损失为:0.5003062121112786
组合7棵回归树后的提升树平方损失为:0.5001010175613262
组合8棵回归树后的提升树平方损失为:0.5000333501953614
组合9棵回归树后的提升树平方损失为:0.5000110130530915
组合10棵回归树后的提升树平方损失为:0.5000036370785973
组合11棵回归树后的提升树平方损失为:0.5000012011836076
组合12棵回归树后的提升树平方损失为:0.500000396707114
组合13棵回归树后的提升树平方损失为:0.5000001310182707
组合14棵回归树后的提升树平方损失为:0.5000000432707248
组合15棵回归树后的提升树平方损失为:0.5000000142908018
组合16棵回归树后的提升树平方损失为:0.5000000047197504
组合17棵回归树后的提升树平方损失为:0.5000000015587684
组合18棵回归树后的提升树平方损失为:0.5000000005148063
组合19棵回归树后的提升树平方损失为:0.5000000001700233
组合20棵回归树后的提升树平方损失为:0.5000000000561523
组合21棵回归树后的提升树平方损失为:0.5000000000185456
组合22棵回归树后的提升树平方损失为:0.500000000006125
组合23棵回归树后的提升树平方损失为:0.500000000002022
组合24棵回归树后的提升树平方损失为:0.5000000000006684
组合25棵回归树后的提升树平方损失为:0.500000000000221
组合26棵回归树后的提升树平方损失为:0.5000000000000726
组合27棵回归树后的提升树平方损失为:0.5000000

 41%|████      | 410/1000 [00:00<00:00, 1337.50it/s]

组合276棵回归树后的提升树平方损失为:0.49999999999999956
组合277棵回归树后的提升树平方损失为:0.5000000000000004
组合278棵回归树后的提升树平方损失为:0.49999999999999956
组合279棵回归树后的提升树平方损失为:0.5000000000000004
组合280棵回归树后的提升树平方损失为:0.49999999999999956
组合281棵回归树后的提升树平方损失为:0.5000000000000004
组合282棵回归树后的提升树平方损失为:0.49999999999999956
组合283棵回归树后的提升树平方损失为:0.5000000000000004
组合284棵回归树后的提升树平方损失为:0.49999999999999956
组合285棵回归树后的提升树平方损失为:0.5000000000000004
组合286棵回归树后的提升树平方损失为:0.49999999999999956
组合287棵回归树后的提升树平方损失为:0.5000000000000004
组合288棵回归树后的提升树平方损失为:0.49999999999999956
组合289棵回归树后的提升树平方损失为:0.5000000000000004
组合290棵回归树后的提升树平方损失为:0.49999999999999956
组合291棵回归树后的提升树平方损失为:0.5000000000000004
组合292棵回归树后的提升树平方损失为:0.49999999999999956
组合293棵回归树后的提升树平方损失为:0.5000000000000004
组合294棵回归树后的提升树平方损失为:0.49999999999999956
组合295棵回归树后的提升树平方损失为:0.5000000000000004
组合296棵回归树后的提升树平方损失为:0.49999999999999956
组合297棵回归树后的提升树平方损失为:0.5000000000000004
组合298棵回归树后的提升树平方损失为:0.49999999999999956
组合299棵回归树后的提升树平方损失为:0.5000000000000004
组合300棵回归树后的提升树平方损失为:0.49999999999999956
组合301棵回归树后的提

 81%|████████  | 811/1000 [00:00<00:00, 1286.99it/s]

组合519棵回归树后的提升树平方损失为:0.5000000000000004
组合520棵回归树后的提升树平方损失为:0.49999999999999956
组合521棵回归树后的提升树平方损失为:0.5000000000000004
组合522棵回归树后的提升树平方损失为:0.49999999999999956
组合523棵回归树后的提升树平方损失为:0.5000000000000004
组合524棵回归树后的提升树平方损失为:0.49999999999999956
组合525棵回归树后的提升树平方损失为:0.5000000000000004
组合526棵回归树后的提升树平方损失为:0.49999999999999956
组合527棵回归树后的提升树平方损失为:0.5000000000000004
组合528棵回归树后的提升树平方损失为:0.49999999999999956
组合529棵回归树后的提升树平方损失为:0.5000000000000004
组合530棵回归树后的提升树平方损失为:0.49999999999999956
组合531棵回归树后的提升树平方损失为:0.5000000000000004
组合532棵回归树后的提升树平方损失为:0.49999999999999956
组合533棵回归树后的提升树平方损失为:0.5000000000000004
组合534棵回归树后的提升树平方损失为:0.49999999999999956
组合535棵回归树后的提升树平方损失为:0.5000000000000004
组合536棵回归树后的提升树平方损失为:0.49999999999999956
组合537棵回归树后的提升树平方损失为:0.5000000000000004
组合538棵回归树后的提升树平方损失为:0.49999999999999956
组合539棵回归树后的提升树平方损失为:0.5000000000000004
组合540棵回归树后的提升树平方损失为:0.49999999999999956
组合541棵回归树后的提升树平方损失为:0.5000000000000004
组合542棵回归树后的提升树平方损失为:0.49999999999999956
组合543棵回归树后的提升树平方损失为:0.5000000000000004
组合544棵回归树后的提升

100%|██████████| 1000/1000 [00:00<00:00, 1312.06it/s]


组合812棵回归树后的提升树平方损失为:0.49999999999999956
组合813棵回归树后的提升树平方损失为:0.5000000000000004
组合814棵回归树后的提升树平方损失为:0.49999999999999956
组合815棵回归树后的提升树平方损失为:0.5000000000000004
组合816棵回归树后的提升树平方损失为:0.49999999999999956
组合817棵回归树后的提升树平方损失为:0.5000000000000004
组合818棵回归树后的提升树平方损失为:0.49999999999999956
组合819棵回归树后的提升树平方损失为:0.5000000000000004
组合820棵回归树后的提升树平方损失为:0.49999999999999956
组合821棵回归树后的提升树平方损失为:0.5000000000000004
组合822棵回归树后的提升树平方损失为:0.49999999999999956
组合823棵回归树后的提升树平方损失为:0.5000000000000004
组合824棵回归树后的提升树平方损失为:0.49999999999999956
组合825棵回归树后的提升树平方损失为:0.5000000000000004
组合826棵回归树后的提升树平方损失为:0.49999999999999956
组合827棵回归树后的提升树平方损失为:0.5000000000000004
组合828棵回归树后的提升树平方损失为:0.49999999999999956
组合829棵回归树后的提升树平方损失为:0.5000000000000004
组合830棵回归树后的提升树平方损失为:0.49999999999999956
组合831棵回归树后的提升树平方损失为:0.5000000000000004
组合832棵回归树后的提升树平方损失为:0.49999999999999956
组合833棵回归树后的提升树平方损失为:0.5000000000000004
组合834棵回归树后的提升树平方损失为:0.49999999999999956
组合835棵回归树后的提升树平方损失为:0.5000000000000004
组合836棵回归树后的提升树平方损失为:0.49999999999999956
组合837棵回归树后的提

100%|██████████| 45/45 [00:00<00:00, 2249.84it/s]

acc 0.022222222222222254
